## Creating an Automated Replinshment Plan

In [140]:
import pandas as pd
from datetime import date
from sqlalchemy import create_engine, text

In [141]:


connection_string = (
    "mssql+pyodbc:///?odbc_connect="
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-PGFBDFD\SQLEXPRESS;"
    "DATABASE=AlgeriaUspaV3T;"
    "Trusted_Connection=yes;"
)

engine = create_engine(connection_string)
df = pd.read_sql("SELECT TOP 10 * FROM ProductAttributes", engine)
print(df)

   ItemTypeCode   ItemCode ProductAtt01 ProductAtt02 ProductAtt03  \
0             1  100021356           01           01            S   
1             1  100021357           01           01            S   
2             1  100021358           01           01            S   
3             1  100021359           01           01            S   
4             1  100021360           01           01            S   
5             1  100021361           01           01            S   
6             1  100021362           01           02            S   
7             1  100021363           01           02            S   
8             1  100021364           01           02            S   
9             1  100021365           01           02            S   

  ProductAtt04 ProductAtt05 ProductAtt06 ProductAtt07 ProductAtt08  ...  \
0                         1                      2020       WINTER  ...   
1                         1                      2020       WINTER  ...   
2              

In [142]:
df = pd.read_sql("SELECT TOP 50 * FROM dbo.RetailSales WHERE OperationDate BETWEEN '2025-06-29' AND '2025-07-06'", engine)
print(df)

    CurrAccTypeCode CurrAccCode OperationDate OperationTime DocumentNumber  \
0                 4       1-4-1    2025-07-04      17:14:01   1-R-7-109191   
1                 4       1-4-1    2025-07-04      17:14:01   1-R-7-109191   
2                 4       1-4-1    2025-07-04      17:09:08   1-R-7-109190   
3                 4       1-4-1    2025-07-04      17:26:47   1-R-7-109196   
4                 4       1-4-1    2025-07-04      17:26:47   1-R-7-109196   
5                 4   1-4-19486    2025-07-04      17:20:54   1-R-7-109193   
6                 4   1-4-19486    2025-07-04      17:20:54   1-R-7-109193   
7                 4       1-4-1    2025-07-04      18:49:24   1-R-7-109211   
8                 4       1-4-1    2025-07-04      20:14:01   1-R-7-109229   
9                 4       1-4-1    2025-07-04      19:59:21   1-R-7-109227   
10                4       1-4-1    2025-07-04      19:59:21   1-R-7-109227   
11                4       1-4-1    2025-07-04      17:28:19   1-

In [172]:
# === Config ===
store_list = ['BEZ', 'CHE', 'AD', 'ORN', 'STF', 'ARD', 'AZ', 'CC']
warehouse_code = 'PRI'
start_date = '2025-07-02'
end_date = '2025-07-06'
plan_date = '2025-07-06'

In [173]:
name_query = text("""
SELECT
    ItemCode,
    ItemDescription
FROM dbo.cdItemDesc                 
""")
name_df = pd.read_sql(name_query, engine)
print(name_df)

       ItemCode ItemDescription
0     100021356           HY001
1     100021357           HY001
2     100021358           HY001
3     100021359           HY001
4     100021360           HY001
...         ...             ...
7089       ExD2     Kur Farkı 3
7090        TZM        Tazminat
7091         VF      Vade Farkı
7092        VF1    Vade Farkı 1
7093        VF2    Vade Farkı 2

[7094 rows x 2 columns]


In [174]:
barcode_query = text("""
SELECT 
    ItemCode,
    ColorCode,
    ItemDim1Code,
    Barcode
FROM dbo.prItemBarcode  
""")
barcode_df = pd.read_sql(barcode_query, engine)
print(barcode_df)

        ItemCode ColorCode ItemDim1Code        Barcode
0      100021356      C044           40  8600000000011
1      100021356      C044           41  8600000000028
2      100021356      C044           42  8600000000035
3      100021356      C044           43  8600000000042
4      100021356      C044           44  8600000000059
...          ...       ...          ...            ...
66421     YP8186         1           56  8681896033765
66422     YP8186         1           58  8681896033789
66423      ZERDA     VR033            0  8682107842800
66424      ZERDA     VR041            0  8682107842824
66425      ZERDA     VR071            0  8682107842817

[66426 rows x 4 columns]


In [175]:
store_list = ['BEZ', 'CHE', 'AD', 'ORN', 'STF', 'ARD', 'AZ', 'CC']
formatted_store_list = ', '.join(f"'{store}'" for store in store_list)

sales_sql = f"""
SELECT 
    StoreCode,
    ItemCode,
    ColorCode,
    ItemDim1Code,
    SUM(CASE WHEN IsReturn = 1 THEN -Qty1 ELSE Qty1 END) AS NetQty
FROM dbo.RetailSales
WHERE StoreCode IN ({formatted_store_list})
  AND OperationDate BETWEEN :start AND :end
GROUP BY StoreCode, ItemCode, ColorCode, ItemDim1Code
HAVING SUM(CASE WHEN IsReturn = 1 THEN -Qty1 ELSE Qty1 END) > 0
"""

# Then run it with only the dates as parameters
sales_df = pd.read_sql(text(sales_sql), engine, params={
    'start': start_date,
    'end': end_date
})


In [176]:
# --- Get warehouse available inventory ---
stock_sql = text("""
SELECT 
    ItemCode,
    ColorCode,
    ItemDim1Code,
    AvailableInventoryQty1
FROM dbo.ItemInventory
WHERE WarehouseCode = :wh
""")
stock_df = pd.read_sql(stock_sql, engine, params={'wh': warehouse_code})
print(stock_df, engine)

          ItemCode ColorCode ItemDim1Code  AvailableInventoryQty1
0        100021356      C044           40                     0.0
1        100021356      C044           41                     0.0
2        100021356      C044           42                     0.0
3        100021356      C044           43                     1.0
4        100021357      C074           40                     0.0
...            ...       ...          ...                     ...
16664  YOCHAPB0135    PB0135           9X                     2.0
16665         YORU     VR013            0                     5.0
16666         YORU     VR028            0                    12.0
16667         YORU     VR033            0                     8.0
16668         YORU     VR183            0                     0.0

[16669 rows x 4 columns] Engine(mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BSERVER%3DDESKTOP-PGFBDFD%5CSQLEXPRESS%3BDATABASE%3DAlgeriaUspaV3T%3BTrusted_Connection%3Dyes%3B)


In [177]:
pri = pd.read_sql("SELECT TOP 50 * FROM dbo.ItemInventory WHERE WarehouseCode LIKE 'PRI' ", engine)
print(pri)

    CompanyCode OfficeCode  StoreTypeCode StoreCode WarehouseCode  \
0           1.0          M              5                     PRI   
1           1.0          M              5                     PRI   
2           1.0          M              5                     PRI   
3           1.0          M              5                     PRI   
4           1.0          M              5                     PRI   
5           1.0          M              5                     PRI   
6           1.0          M              5                     PRI   
7           1.0          M              5                     PRI   
8           1.0          M              5                     PRI   
9           1.0          M              5                     PRI   
10          1.0          M              5                     PRI   
11          1.0          M              5                     PRI   
12          1.0          M              5                     PRI   
13          1.0          M        

In [178]:
replenishment_results = []

for store in store_list:
    store_sales = sales_df[sales_df['StoreCode'] == store]

    merged = pd.merge(
        store_sales,
        stock_df,
        on=['ItemCode', 'ColorCode', 'ItemDim1Code'],
        how='inner'
    )

    merged['ToTransfer'] = merged[['NetQty', 'AvailableInventoryQty1']].min(axis=1)
    plan_df = merged[merged['ToTransfer'] > 0].copy()

    plan_df['StoreID'] = store
    plan_df['PlanDate'] = plan_date
    plan_df['SourceWarehouse'] = warehouse_code
    plan_df.rename(columns={'ToTransfer': 'Quantity'}, inplace=True)

    replenishment_results.append(
        plan_df[['StoreID', 'ItemCode', 'ColorCode', 'ItemDim1Code', 'Quantity', 'PlanDate', 'SourceWarehouse']]
    )

# Combine all stores into one DataFrame
final_replenishment = pd.concat(replenishment_results, ignore_index=True)


In [179]:
pivot_df = final_replenishment.pivot_table(
    index=['ItemCode', 'ColorCode', 'ItemDim1Code'],   # Group by product
    columns='StoreID',                                 # Store codes as columns
    values='Quantity',                                 # Quantity as values
    aggfunc='sum',                                     # In case of duplicates
    fill_value=0                                       # Fill blanks with 0
)

# Reset index to make it a flat table
pivot_df = pivot_df.reset_index()

# Optionally add a total column
# Only sum columns that exist in pivot_df
existing_stores = [col for col in store_list if col in pivot_df.columns]
pivot_df['Total'] = pivot_df[existing_stores].sum(axis=1)


# Sort by total or item
pivot_df = pivot_df.sort_values(by='Total', ascending=False)

# Save to Excel

print(pivot_df, engine)

StoreID     ItemCode ColorCode ItemDim1Code   AD  ARD   AZ  BEZ   CC  CHE  \
229          2085130     VR033            L  2.0  0.0  0.0  2.0  0.0  2.0   
147          2084410     VR079            L  0.0  1.0  0.0  1.0  0.0  0.0   
297      EARLES-IY24     VR086            0  0.0  0.0  0.0  2.0  0.0  3.0   
296      EARLES-IY24     VR046            0  0.0  0.0  0.0  5.0  0.0  2.0   
237          2085130     VR049            L  1.0  2.0  0.0  0.0  0.0  1.0   
..               ...       ...          ...  ...  ...  ...  ...  ...  ...   
313        US1670-V1      C073          3_4  1.0  0.0  0.0  0.0  0.0  0.0   
312        US1435-V1     C0305        10_11  0.0  0.0  1.0  0.0  0.0  0.0   
311        US1377-V1      C038          3_4  1.0  0.0  0.0  0.0  0.0  0.0   
310        US1372-V2      C038        13_14  0.0  0.0  0.0  0.0  0.0  0.0   
1            1358148     VR085           31  1.0  0.0  0.0  0.0  0.0  0.0   

StoreID  ORN  STF  Total  
229      2.0  1.0    9.0  
147      3.0  3.0    

In [180]:
# 1. Make sure store columns exist in pivot_df
for store in store_list:
    if store not in pivot_df.columns:
        pivot_df[store] = 0

# 2. Merge starting inventory correctly
# First ensure all key columns have same types
stock_df['ItemDim1Code'] = stock_df['ItemDim1Code'].astype(str)
pivot_df['ItemDim1Code'] = pivot_df['ItemDim1Code'].astype(str)

# Merge
stock_summary = stock_df[['ItemCode', 'ColorCode', 'ItemDim1Code', 'AvailableInventoryQty1']]
stock_summary = stock_summary.rename(columns={'AvailableInventoryQty1': 'StartingWarehouseQty'})

pivot_df = pd.merge(
    pivot_df,
    stock_summary,
    on=['ItemCode', 'ColorCode', 'ItemDim1Code'],
    how='left'
)

# 3. Fill any missing inventory values with 0
pivot_df['StartingWarehouseQty'] = pivot_df['StartingWarehouseQty'].fillna(0)

# 4. Compute remaining quantity
pivot_df['Total'] = pivot_df[store_list].sum(axis=1)
pivot_df['RemainingWarehouseQty'] = pivot_df['StartingWarehouseQty'] - pivot_df['Total']
pivot_df['RemainingWarehouseQty'] = pivot_df['RemainingWarehouseQty'].clip(lower=0)


In [181]:
pivot_df = pivot_df[[
    'ItemCode', 'ColorCode', 'ItemDim1Code',
    *store_list, 'Total', 'StartingWarehouseQty', 'RemainingWarehouseQty'
]]
print(pivot_df, engine)

        ItemCode ColorCode ItemDim1Code  BEZ  CHE   AD  ORN  STF  ARD   AZ  \
0        2085130     VR033            L  2.0  2.0  2.0  2.0  1.0  0.0  0.0   
1        2084410     VR079            L  1.0  0.0  0.0  3.0  3.0  1.0  0.0   
2    EARLES-IY24     VR086            0  2.0  3.0  0.0  1.0  2.0  0.0  0.0   
3    EARLES-IY24     VR046            0  5.0  2.0  0.0  0.0  0.0  0.0  0.0   
4        2085130     VR049            L  0.0  1.0  1.0  1.0  1.0  2.0  0.0   
..           ...       ...          ...  ...  ...  ...  ...  ...  ...  ...   
320    US1670-V1      C073          3_4  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
321    US1435-V1     C0305        10_11  0.0  0.0  0.0  0.0  0.0  0.0  1.0   
322    US1377-V1      C038          3_4  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
323    US1372-V2      C038        13_14  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
324      1358148     VR085           31  0.0  0.0  1.0  0.0  0.0  0.0  0.0   

      CC  Total  StartingWarehouseQty  RemainingWarehouseQty  


In [182]:
# Ordering the result by itemcode then color code
pivot_df['ItemCode'] = pivot_df['ItemCode'].astype(str)
pivot_df['ColorCode'] = pivot_df['ColorCode'].astype(str)
pivot_df = pivot_df.sort_values(by=['ItemCode', 'ColorCode'])

In [183]:
item_inventory_df = pd.read_sql(
    text("SELECT * FROM dbo.ItemInventory"),
    engine
)
item_inventory_df['ItemDim1Code'] = item_inventory_df['ItemDim1Code'].astype(str)

In [184]:


# Filter for inventory rows where StoreCode matches a store (not warehouse stock)
store_inv_df = item_inventory_df[
    item_inventory_df['StoreCode'].isin(store_list) &
    item_inventory_df['WarehouseCode'].isin(store_list)  # This excludes warehouse rows
]


In [185]:
# Pivot so that each store has its own inventory column
store_inv_pivot = store_inv_df.pivot_table(
    index=['ItemCode', 'ColorCode', 'ItemDim1Code'],
    columns='StoreCode',
    values='AvailableInventoryQty1',
    aggfunc='sum',
    fill_value=0
)

# Rename columns to something like 'BEZ_Inv'
store_inv_pivot.columns = [f"{col}_Inv" for col in store_inv_pivot.columns]
store_inv_pivot = store_inv_pivot.reset_index()


In [186]:
pivot_df = pd.merge(
    pivot_df,
    store_inv_pivot,
    on=['ItemCode', 'ColorCode', 'ItemDim1Code'],
    how='left'
)


In [187]:
columns_order = ['ItemCode', 'ColorCode', 'ItemDim1Code']
for store in store_list:
    columns_order += [f'{store}_Inv', store]
columns_order += ['Total', 'StartingWarehouseQty', 'RemainingWarehouseQty']

pivot_df = pivot_df[columns_order]
print(pivot_df, engine)

       ItemCode ColorCode ItemDim1Code  BEZ_Inv  BEZ  CHE_Inv  CHE  AD_Inv  \
0     101156651      C074           35      0.0  0.0      0.0  0.0     0.0   
1       1358148     VR085           31      0.0  0.0      0.0  0.0     1.0   
2       1359493     VR027          4_5      2.0  0.0      0.0  0.0     2.0   
3       1360141     VR027            L      0.0  0.0      2.0  1.0     0.0   
4       1361739     VR036            M      0.0  0.0      0.0  0.0     2.0   
..          ...       ...          ...      ...  ...      ...  ...     ...   
320  USB2004-V1     C0449        18_24      0.0  0.0      0.0  0.0     1.0   
321  USB2004-V1     C0449        12_18      0.0  0.0      0.0  0.0     1.0   
322  USB2100-V1     C0141          0_3      0.0  0.0      1.0  0.0     0.0   
323  USB2106-V1     C0141          0_3      0.0  1.0      1.0  0.0     2.0   
324        YORU     VR033            0      2.0  0.0      0.0  0.0     0.0   

      AD  ORN_Inv  ...  STF  ARD_Inv  ARD  AZ_Inv   AZ  CC_Inv 

In [188]:
result_with_barcode = pd.merge(
    pivot_df,
    barcode_df,
    on=['ItemCode', 'ColorCode', 'ItemDim1Code'],
    how='left'  # Use left join to preserve all replenishment rows
)


In [189]:
result_with_name = pd.merge(
    result_with_barcode,
    name_df,
    on = ['ItemCode'],
    how = 'left'
)
print(result_with_name)

       ItemCode ColorCode ItemDim1Code  BEZ_Inv  BEZ  CHE_Inv  CHE  AD_Inv  \
0     101156651      C074           35      0.0  0.0      0.0  0.0     0.0   
1       1358148     VR085           31      0.0  0.0      0.0  0.0     1.0   
2       1359493     VR027          4_5      2.0  0.0      0.0  0.0     2.0   
3       1360141     VR027            L      0.0  0.0      2.0  1.0     0.0   
4       1361739     VR036            M      0.0  0.0      0.0  0.0     2.0   
..          ...       ...          ...      ...  ...      ...  ...     ...   
321  USB2004-V1     C0449        18_24      0.0  0.0      0.0  0.0     1.0   
322  USB2004-V1     C0449        12_18      0.0  0.0      0.0  0.0     1.0   
323  USB2100-V1     C0141          0_3      0.0  0.0      1.0  0.0     0.0   
324  USB2106-V1     C0141          0_3      0.0  1.0      1.0  0.0     2.0   
325        YORU     VR033            0      2.0  0.0      0.0  0.0     0.0   

      AD  ORN_Inv  ...  ARD  AZ_Inv   AZ  CC_Inv   CC  Total  \

In [190]:
attribute_query = text("""
SELECT 
    ItemCode, 
    GenderCode = ProductAtt02, 
    SubCategoryCode = ProductAtt14
FROM dbo.ProductAttributes                
""")
attribute_df = pd.read_sql(attribute_query, engine)
print(attribute_df)
#cdItemAttributeDesc

result_with_attribute = pd.merge(
    result_with_name,
    attribute_df,
    on = ['ItemCode'],
    how = 'left'
)
print(result_with_attribute)


       ItemCode GenderCode SubCategoryCode
0     100021356         01          CA0025
1     100021357         01          CA0025
2     100021358         01          CA0025
3     100021359         01          CA0025
4     100021360         01          CA0025
...         ...        ...             ...
7988     YP8065         01         CA00180
7989     YP8086         01         CA00180
7990     YP8089         01         CA00180
7991     YP8186         01         CA00180
7992      ZERDA         02          CA0026

[7993 rows x 3 columns]
       ItemCode ColorCode ItemDim1Code  BEZ_Inv  BEZ  CHE_Inv  CHE  AD_Inv  \
0     101156651      C074           35      0.0  0.0      0.0  0.0     0.0   
1       1358148     VR085           31      0.0  0.0      0.0  0.0     1.0   
2       1359493     VR027          4_5      2.0  0.0      0.0  0.0     2.0   
3       1360141     VR027            L      0.0  0.0      2.0  1.0     0.0   
4       1361739     VR036            M      0.0  0.0      0.0  0.0   

In [191]:
# Get Gender
gender_query = text("""
SELECT 
   GenderCode = AttributeCode,
   GenderDescription = AttributeDescription 
    
FROM dbo.cdItemAttributeDesc
WHERE AttributeTypeCode LIKE '2'
""")
gender_df = pd.read_sql(gender_query, engine)
print(gender_df)


  GenderCode GenderDescription
0         01               MAN
1         02             WOMAN
2         03               BOY
3         04              GIRL
4         05              BABY
5         06            UNISEX


In [192]:
# Get SubCategory
subcat_query = text("""
SELECT 
   SubCategoryCode = AttributeCode,
   SubCategoryDescription = AttributeDescription 
    
FROM dbo.cdItemAttributeDesc
WHERE AttributeTypeCode LIKE '14'
""")
subcat_df = pd.read_sql(subcat_query, engine)
print(subcat_df)

    SubCategoryCode SubCategoryDescription
0            CA0010                   BUST
1           CA00100                 PNCH-T
2           CA00101               PNT-CR-J
3           CA00102             PNT-CR-JOG
4           CA00103               PNT-CR-T
..              ...                    ...
186          CA0096                 PANT-V
187          CA0097                    PJM
188          CA0098                   PL-L
189          CA0099                  PL-TR
190           CA022           CHINOS BASIC

[191 rows x 2 columns]


In [193]:
result_with_genderdesc = pd.merge(
    result_with_attribute,
    gender_df,
    on = ['GenderCode'],
    how = 'left'
)
print(result_with_genderdesc)

       ItemCode ColorCode ItemDim1Code  BEZ_Inv  BEZ  CHE_Inv  CHE  AD_Inv  \
0     101156651      C074           35      0.0  0.0      0.0  0.0     0.0   
1       1358148     VR085           31      0.0  0.0      0.0  0.0     1.0   
2       1359493     VR027          4_5      2.0  0.0      0.0  0.0     2.0   
3       1360141     VR027            L      0.0  0.0      2.0  1.0     0.0   
4       1361739     VR036            M      0.0  0.0      0.0  0.0     2.0   
..          ...       ...          ...      ...  ...      ...  ...     ...   
321  USB2004-V1     C0449        18_24      0.0  0.0      0.0  0.0     1.0   
322  USB2004-V1     C0449        12_18      0.0  0.0      0.0  0.0     1.0   
323  USB2100-V1     C0141          0_3      0.0  0.0      1.0  0.0     0.0   
324  USB2106-V1     C0141          0_3      0.0  1.0      1.0  0.0     2.0   
325        YORU     VR033            0      2.0  0.0      0.0  0.0     0.0   

      AD  ORN_Inv  ...  CC_Inv   CC  Total  StartingWarehouseQt

In [194]:
result_with_subcatdesc = pd.merge(
    result_with_genderdesc,
    subcat_df,
    on = ['SubCategoryCode'],
    how = 'left'
)
print(result_with_subcatdesc)

       ItemCode ColorCode ItemDim1Code  BEZ_Inv  BEZ  CHE_Inv  CHE  AD_Inv  \
0     101156651      C074           35      0.0  0.0      0.0  0.0     0.0   
1       1358148     VR085           31      0.0  0.0      0.0  0.0     1.0   
2       1359493     VR027          4_5      2.0  0.0      0.0  0.0     2.0   
3       1360141     VR027            L      0.0  0.0      2.0  1.0     0.0   
4       1361739     VR036            M      0.0  0.0      0.0  0.0     2.0   
..          ...       ...          ...      ...  ...      ...  ...     ...   
321  USB2004-V1     C0449        18_24      0.0  0.0      0.0  0.0     1.0   
322  USB2004-V1     C0449        12_18      0.0  0.0      0.0  0.0     1.0   
323  USB2100-V1     C0141          0_3      0.0  0.0      1.0  0.0     0.0   
324  USB2106-V1     C0141          0_3      0.0  1.0      1.0  0.0     2.0   
325        YORU     VR033            0      2.0  0.0      0.0  0.0     0.0   

      AD  ORN_Inv  ...   CC  Total  StartingWarehouseQty  \
0  

In [195]:
#cdColorDesc
# Get Color Description
color_query = text("""
SELECT 
   ColorCode = ColorCode,
   ColorDescription = ColorDescription 
    
FROM dbo.cdColorDesc

""")
color_df = pd.read_sql(color_query, engine)
print(color_df)


    ColorCode ColorDescription
0         001            BLACK
1           1            black
2         100             GREY
3         110       GREY LIGHT
4        1190  NAVY BLUE-WHITE
..        ...              ...
918    YS0931      GREEN D'EAU
919    YS0995            GREEN
920    YS0997      GREEN FONCE
921    YS1003             BLUE
922    YS1004              RED

[923 rows x 2 columns]


In [196]:
result_with_color = pd.merge(
    result_with_subcatdesc,
    color_df,
    on = ['ColorCode'],
    how = 'left'
)
print(result_with_color)


       ItemCode ColorCode ItemDim1Code  BEZ_Inv  BEZ  CHE_Inv  CHE  AD_Inv  \
0     101156651      C074           35      0.0  0.0      0.0  0.0     0.0   
1       1358148     VR085           31      0.0  0.0      0.0  0.0     1.0   
2       1359493     VR027          4_5      2.0  0.0      0.0  0.0     2.0   
3       1360141     VR027            L      0.0  0.0      2.0  1.0     0.0   
4       1361739     VR036            M      0.0  0.0      0.0  0.0     2.0   
..          ...       ...          ...      ...  ...      ...  ...     ...   
321  USB2004-V1     C0449        18_24      0.0  0.0      0.0  0.0     1.0   
322  USB2004-V1     C0449        12_18      0.0  0.0      0.0  0.0     1.0   
323  USB2100-V1     C0141          0_3      0.0  0.0      1.0  0.0     0.0   
324  USB2106-V1     C0141          0_3      0.0  1.0      1.0  0.0     2.0   
325        YORU     VR033            0      2.0  0.0      0.0  0.0     0.0   

      AD  ORN_Inv  ...  Total  StartingWarehouseQty  RemainingW

In [197]:
reassortbehindscene_df = result_with_color[['ItemCode',
                            'ColorCode',
                            'ColorDescription',
                            'ItemDescription',
                            'ItemDim1Code',
                            'GenderDescription',
                            'SubCategoryDescription',
                            'Barcode',
                            'RemainingWarehouseQty',
                            'BEZ','BEZ_Inv','CHE','CHE_Inv','AD','AD_Inv','CC','CC_Inv','STF','STF_Inv','ORN','ORN_Inv','AZ','AZ_Inv','ARD','ARD_Inv',
                            'Total',
                            ]].rename(columns={
    'ItemDim1Code': 'Size',
    'GenderDescription' : 'Gender',
    'RemainingWarehouseQty': 'reste',
    'SubCategoryDescription': 'Category',
    'ItemDescription': 'Name'
    
})

In [198]:
reassort_df = result_with_color[['ItemCode',
                            'ColorCode',
                            'ColorDescription',
                            'ItemDescription',
                            'ItemDim1Code',
                            'GenderDescription',
                            'SubCategoryDescription',
                            'Barcode',
                            'RemainingWarehouseQty',
                            'BEZ','CHE','AD','CC','STF','ORN','AZ','ARD',
                            'Total',
                            ]].rename(columns={
    'ItemDim1Code': 'Size',
    'GenderDescription' : 'Gender',
    'RemainingWarehouseQty': 'reste',
    'SubCategoryDescription': 'Category',
    'ItemDescription': 'Name'
    
})

In [199]:
# Export to Excel

reassort_df.to_excel(
    f"C:/Users/ACER/Downloads/réassort_{plan_date}_{warehouse_code}.xlsx",
    sheet_name='Réassort',
    index=False
)
reassortbehindscene_df.to_excel(
    f"C:/Users/ACER/Downloads/réassort_{plan_date}_{warehouse_code} making of.xlsx",
    sheet_name='Réassort',
    index=False
)